# 01 Spatial Validation for Early Voting Locations

**Purpose**: Validate Google Maps geocoding results against province boundaries and generate fallback coordinates for failed locations.

**Tech Summary**:
- **Input**: 
  - `../intermediate/early_voting_geocoded_raw.parquet` (Google Maps geocoding results)
  - `/home/ben/ddd/ninyawee/landmap/web/static/data/tha_admin2.geojson` (Province/district boundaries)
  - `/home/ben/ddd/ninyawee/Thai-ECT-election-map/ECT Constituencies/2569/ShapeFile/2569_Election_Constituencies.shp` (ECT electoral districts 2569)
- **Process**:
  - Load geocoded results
  - Point-in-polygon test against province boundaries
  - Generate random points within constituency for failed validations
  - Assign quality tiers (A+ = validated, D = synthetic)
- **Output**: `../intermediate/early_voting_validated.parquet`
- **Dependencies**: geopandas, shapely, pydantic

---

In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import random
import sys

sys.path.insert(0, "..")
from lib.models import GMapEntry

## Load Geocoded Results

In [2]:
# Load geocoded early voting locations
df = pd.read_parquet("../intermediate/early_voting_geocoded_raw.parquet")
print(f"Loaded {len(df):,} locations")
df.head()

Loaded 424 locations


,location_name,location_type,area_prefix,buildings,floor,extra_info,subdistrict,district,original,geocode_query,GMap,GMapLen
0,สำนักงานเขตสัมพันธวงศ์,government_office,บริเวณ,[],None,None,ตลาดน้อย,สัมพันธวงศ์,บริเวณสำนักงานเขตสัมพันธวงศ์ แขวงตลาดน้อย,สำนักงานเขตสัมพันธวงศ์,"[{'address_components': [{'long_name': '37', '...",1
1,สำนักงานเขตป้อมปราบศัตรูพ่าย,government_office,บริเวณ,[],None,None,วัดโสมนัส,None,บริเวณสำนักงานเขตป้อมปราบศัตรูพ่าย แขวงวัดโสมนัส,สำนักงานเขตป้อมปราบศัตรูพ่าย,"[{'address_components': [{'long_name': '50', '...",1
2,สำนักงานเขตพระนคร,government_office,บริเวณลานจอดรถ,[],None,None,วัดสามพระยา,พระนคร,บริเวณลานจอดรถสำนักงานเขตพระนคร แขวงวัดสามพระยา,สำนักงานเขตพระนคร,"[{'address_components': [{'long_name': '78', '...",1
3,สมาคมใหหนำแห่งประเทศไทย,other,บริเวณ,[],None,None,สี่พระยา,None,บริเวณสโมสรสมาคมใหหนำแห่งประเทศไทย แขวงสี่พระยา,บริเวณสโมสรสมาคมใหหนำแห่งประเทศไทย,"[{'address_components': [{'long_name': '324', ...",1
4,วชิราวุธวิทยาลัย,school,บริเวณภายใน,[],None,None,ดุสิต,None,บริเวณภายในวชิราวุธวิทยาลัย แขวงดุสิต,วชิราวุธวิทยาลัย,"[{'address_components': [{'long_name': '197', ...",1


In [3]:
# Parse Google Maps results into GMapEntry objects
def parse_gmap_results(gmap_list):
    return [GMapEntry.from_geocode_result(r) for r in gmap_list]


df["GMapObjs"] = df["GMap"].apply(parse_gmap_results)
df["GMapLen"] = df["GMapObjs"].apply(len)

print("Geocoding results distribution:")
print(df["GMapLen"].value_counts().sort_index())

Geocoding results distribution:
GMapLen
1    401
2     22
3      1
Name: count, dtype: int64


## Load Validation Boundaries

In [4]:
# Load province boundaries (admin2 = districts, contains province info)
admin2 = gpd.read_file(
    "/home/ben/ddd/ninyawee/landmap/web/static/data/tha_admin2.geojson"
)
admin2 = admin2.to_crs(epsg=4326)
print(f"Loaded {len(admin2)} districts")
admin2.head()

Loaded 928 districts


,adm2_name,adm2_name1,adm2_name2,adm2_name3,adm2_pcode,adm1_name,adm1_name1,adm1_name2,adm1_name3,adm1_pcode,...,area_sqkm,version,lang,lang1,lang2,lang3,adm2_ref_name,center_lat,center_lon,geometry
0,Phra Nakhon,พระนคร,None,None,TH1001,Bangkok,กรุงเทพมหานคร,None,None,TH10,...,5.389902,v01,en,th,None,None,Phra Nakhon,13.755672,100.496883,"POLYGON ((100.50075 13.74107, 100.50093 13.741..."
1,Dusit,ดุสิต,None,None,TH1002,Bangkok,กรุงเทพมหานคร,None,None,TH10,...,11.367963,v01,en,th,None,None,Dusit,13.776479,100.514626,"POLYGON ((100.52 13.8, 100.51945 13.80012, 100..."
2,Nong Chok,หนองจอก,None,None,TH1003,Bangkok,กรุงเทพมหานคร,None,None,TH10,...,237.516621,v01,en,th,None,None,Nong Chok,13.842266,100.858234,"POLYGON ((100.91398 13.94621, 100.91397 13.946..."
3,Bang Rak,บางรัก,None,None,TH1004,Bangkok,กรุงเทพมหานคร,None,None,TH10,...,4.032189,v01,en,th,None,None,Bang Rak,13.728179,100.526157,"POLYGON ((100.51703 13.71808, 100.51723 13.718..."
4,Bang Khen,บางเขน,None,None,TH1005,Bangkok,กรุงเทพมหานคร,None,None,TH10,...,40.840932,v01,en,th,None,None,Bang Khen,13.871248,100.636296,"POLYGON ((100.60999 13.89078, 100.61007 13.890..."


In [5]:
# Check column names for province
print(admin2.columns.tolist())

['adm2_name', 'adm2_name1', 'adm2_name2', 'adm2_name3', 'adm2_pcode', 'adm1_name', 'adm1_name1', 'adm1_name2', 'adm1_name3', 'adm1_pcode', 'adm0_name', 'adm0_name1', 'adm0_name2', 'adm0_name3', 'adm0_pcode', 'valid_on', 'valid_to', 'area_sqkm', 'version', 'lang', 'lang1', 'lang2', 'lang3', 'adm2_ref_name', 'center_lat', 'center_lon', 'geometry']


In [6]:
# Load ECT 2569 electoral constituencies
ect = gpd.read_file(
    "/home/ben/ddd/ninyawee/Thai-ECT-election-map/ECT Constituencies/2569/ShapeFile/2569_Election_Constituencies.shp"
)
ect = ect.to_crs(epsg=4326)
print(f"Loaded {len(ect)} constituencies")
ect.head()

Loaded 400 constituencies


,P_name,CONS_no,geometry
0,กำแพงเพชร,4,"POLYGON ((99.8067 15.94157, 99.80656 15.94095,..."
1,สงขลา,1,"MULTIPOLYGON (((100.66906 7.07616, 100.66929 7..."
2,กำแพงเพชร,3,"POLYGON ((99.42389 16.14629, 99.42514 16.14636..."
3,สงขลา,2,"POLYGON ((100.49797 7.01946, 100.49944 7.01881..."
4,กำแพงเพชร,2,"POLYGON ((99.6744 16.54939, 99.67401 16.54715,..."


In [7]:
# Check column names
print(ect.columns.tolist())

['P_name', 'CONS_no', 'geometry']


## Load Source Data for Province/Constituency Matching

In [8]:
# Load source data to get province and constituency info
source_df = pd.read_csv("../inputs/vote69_early_voting_เลือกตั้งล่วงหน้า.csv")
print(f"Source file has {len(source_df)} rows")
source_df.head()

Source file has 424 rows


,จังหวัด,เขตเลือกตั้ง,เขต,สถานที่เลือกตั้งกลาง,รวมจำนวน,จำนวนชุด,อาคาร,จำนวนเต็นท์,การเดินทาง.ที่จอดรถ,เดินทาง.รถไฟฟ้าใต้ดิน,การเดินทาง.รถโดยสารประจำทาง,การเดินทาง.เรือโดยสาร,การเดินทาง.note,การเดินทาง.images,แผนผังการเลือกตั้ง.images,แผนผังการเลือกตั้ง.clips
0,กรุงเทพมหานคร,1,สัมพันธวงศ์,บริเวณสำนักงานเขตสัมพันธวงศ์ แขวงตลาดน้อย,2305,5,0,10,NaN,NaN,NaN,NaN,NaN,กรุงเทพ_สัมพันธวงศ์_1.jpg,NaN,NaN
1,กรุงเทพมหานคร,1,ป้อมปราบศัตรูพ่าย,บริเวณสำนักงานเขตป้อมปราบศัตรูพ่าย แขวงวัดโสมนัส,3997,8,0,16,NaN,NaN,NaN,NaN,NaN,กรุงเทพ_ป้อมปราบศัตรูพ่าย_1.jpg,NaN,NaN
2,กรุงเทพมหานคร,1,พระนคร,บริเวณลานจอดรถสำนักงานเขตพระนคร แขวงวัดสามพระยา,5705,11,0,22,NaN,NaN,NaN,NaN,NaN,กรุงเทพ_พระนคร_1.jpg,NaN,NaN
3,กรุงเทพมหานคร,1,บางรัก,บริเวณสโมสรสมาคมใหหนำแห่งประเทศไทย แขวงสี่พระยา,6889,14,0,28,NaN,NaN,NaN,NaN,NaN,กรุงเทพ_บางรัก_1.jpg,NaN,NaN
4,กรุงเทพมหานคร,1,ดุสิต,บริเวณภายในวชิราวุธวิทยาลัย แขวงดุสิต,11785,24,0,48,NaN,NaN,NaN,NaN,NaN,กรุงเทพ_ดุสิต_1.jpg,NaN,NaN


In [9]:
# Create lookup from original location name to province/constituency
source_lookup = source_df[["จังหวัด", "เขตเลือกตั้ง", "สถานที่เลือกตั้งกลาง"]].copy()
source_lookup.columns = ["province", "constituency_no", "original"]

# Clean province name (remove จังหวัด prefix if present)
source_lookup["province_clean"] = source_lookup["province"].str.removeprefix("จังหวัด")

# Merge with geocoded data
df = df.merge(source_lookup, on="original", how="left")
df.head()

,location_name,location_type,area_prefix,buildings,floor,extra_info,subdistrict,district,original,geocode_query,GMap,GMapLen,GMapObjs,province,constituency_no,province_clean
0,สำนักงานเขตสัมพันธวงศ์,government_office,บริเวณ,[],None,None,ตลาดน้อย,สัมพันธวงศ์,บริเวณสำนักงานเขตสัมพันธวงศ์ แขวงตลาดน้อย,สำนักงานเขตสัมพันธวงศ์,"[{'address_components': [{'long_name': '37', '...",1,[lat=13.7315476 lng=100.5139127 place_id='ChIJ...,กรุงเทพมหานคร,1,กรุงเทพมหานคร
1,สำนักงานเขตป้อมปราบศัตรูพ่าย,government_office,บริเวณ,[],None,None,วัดโสมนัส,None,บริเวณสำนักงานเขตป้อมปราบศัตรูพ่าย แขวงวัดโสมนัส,สำนักงานเขตป้อมปราบศัตรูพ่าย,"[{'address_components': [{'long_name': '50', '...",1,[lat=13.7581428 lng=100.5130792 place_id='ChIJ...,กรุงเทพมหานคร,1,กรุงเทพมหานคร
2,สำนักงานเขตพระนคร,government_office,บริเวณลานจอดรถ,[],None,None,วัดสามพระยา,พระนคร,บริเวณลานจอดรถสำนักงานเขตพระนคร แขวงวัดสามพระยา,สำนักงานเขตพระนคร,"[{'address_components': [{'long_name': '78', '...",1,[lat=13.7648878 lng=100.4987564 place_id='ChIJ...,กรุงเทพมหานคร,1,กรุงเทพมหานคร
3,สมาคมใหหนำแห่งประเทศไทย,other,บริเวณ,[],None,None,สี่พระยา,None,บริเวณสโมสรสมาคมใหหนำแห่งประเทศไทย แขวงสี่พระยา,บริเวณสโมสรสมาคมใหหนำแห่งประเทศไทย,"[{'address_components': [{'long_name': '324', ...",1,[lat=13.7265678 lng=100.5196814 place_id='ChIJ...,กรุงเทพมหานคร,1,กรุงเทพมหานคร
4,วชิราวุธวิทยาลัย,school,บริเวณภายใน,[],None,None,ดุสิต,None,บริเวณภายในวชิราวุธวิทยาลัย แขวงดุสิต,วชิราวุธวิทยาลัย,"[{'address_components': [{'long_name': '197', ...",1,[lat=13.7768635 lng=100.51575 place_id='ChIJd2...,กรุงเทพมหานคร,1,กรุงเทพมหานคร


## Create Province Polygons (Dissolve Districts)

In [10]:
# Dissolve districts into province polygons
# First check the province name column
province_col = "adm1_name"  # Adjust based on actual column
provinces = admin2.dissolve(by=province_col).reset_index()
provinces = provinces[[province_col, "geometry"]]
provinces.columns = ["province_name", "geometry"]
print(f"Created {len(provinces)} province polygons")
provinces.head()

Created 77 province polygons


,province_name,geometry
0,Amnat Charoen,"POLYGON ((104.9039 15.74851, 104.90376 15.7482..."
1,Ang Thong,"POLYGON ((100.49241 14.53209, 100.49249 14.531..."
2,Bangkok,"POLYGON ((100.51739 13.66079, 100.51737 13.660..."
3,Bueng Kan,"POLYGON ((103.99925 17.89309, 103.99923 17.893..."
4,Buri Ram,"POLYGON ((102.81314 14.16266, 102.81298 14.161..."


## Match to ECT Constituency Polygons

In [11]:
# Merge with ECT constituency polygons
# ECT uses P_name for province and CONS_no for constituency number
df = df.merge(
    ect[["P_name", "CONS_no", "geometry"]],
    left_on=["province_clean", "constituency_no"],
    right_on=["P_name", "CONS_no"],
    how="left",
    suffixes=("", "_ect"),
)
df = df.rename(columns={"geometry": "constituency_geom"})
df = df.drop(columns=["P_name", "CONS_no"], errors="ignore")

# Check if we have constituency geometry for all
print(f"Rows without constituency geometry: {df['constituency_geom'].isna().sum()}")

Rows without constituency geometry: 0


## Validate Points Within Constituency

In [12]:
# Filter geocoded points that fall within their constituency
def filter_points_in_polygon(row):
    """Keep only geocoded points that fall within the constituency polygon."""
    if row["constituency_geom"] is None:
        return []

    valid_points = []
    for obj in row["GMapObjs"]:
        if obj.point.within(row["constituency_geom"]):
            valid_points.append(obj)
    return valid_points


df["GMapObjsFiltered"] = df.apply(filter_points_in_polygon, axis=1)
df["GMapObjsFilteredLen"] = df["GMapObjsFiltered"].apply(len)

print("Validated points distribution:")
print(df["GMapObjsFilteredLen"].value_counts().sort_index())

Validated points distribution:
GMapObjsFilteredLen
0     51
1    356
2     17
Name: count, dtype: int64


## Generate Random Fallback Points

In [13]:
def gen_random_point_in_polygon(poly):
    """Generate a random point within a polygon."""
    if poly is None:
        return None
    minx, miny, maxx, maxy = poly.bounds
    for _ in range(1000):  # Max attempts
        p = Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
        if poly.contains(p):
            return p
    # Fallback to centroid
    return poly.centroid


df["RandomPoint"] = df["constituency_geom"].apply(gen_random_point_in_polygon)

In [14]:
# Select best point: validated geocode or random fallback
# Output separate clear flags:
#   geocoded: bool - did Google Maps return results?
#   within_boundary: bool - is the geocoded point within the constituency polygon?


def select_best_point(row):
    geocoded = row["GMapLen"] > 0
    within_boundary = len(row["GMapObjsFiltered"]) > 0

    if within_boundary:
        g = row["GMapObjsFiltered"][0]
        return {
            "Lat": g.lat,
            "Lng": g.lng,
            "PlaceId": g.place_id,
            "FormattedAddress": g.formatted_address,
            "geocoded": True,
            "within_boundary": True,
        }
    elif row["RandomPoint"] is not None:
        # Use first geocoded point coords if available (even though outside boundary)
        if geocoded:
            g = row["GMapObjs"][0]
            lat, lng = g.lat, g.lng
            place_id = g.place_id
            formatted_address = g.formatted_address
        else:
            lat, lng = row["RandomPoint"].y, row["RandomPoint"].x
            place_id = ""
            formatted_address = ""
        return {
            "Lat": lat,
            "Lng": lng,
            "PlaceId": place_id,
            "FormattedAddress": formatted_address,
            "geocoded": geocoded,
            "within_boundary": False,
        }
    else:
        return {
            "Lat": None,
            "Lng": None,
            "PlaceId": "",
            "FormattedAddress": "",
            "geocoded": geocoded,
            "within_boundary": False,
        }


result_cols = pd.DataFrame(df.apply(select_best_point, axis=1).tolist(), index=df.index)
df = pd.concat([df, result_cols], axis=1)

In [15]:
# Summary
total = len(df)
geocoded_count = df["geocoded"].sum()
within_count = df["within_boundary"].sum()

print(f"Total locations: {total}")
print(f"  geocoded=True:        {geocoded_count} ({geocoded_count / total * 100:.1f}%)")
print(f"  within_boundary=True: {within_count} ({within_count / total * 100:.1f}%)")
print("\nBreakdown:")
print(df.groupby(["geocoded", "within_boundary"]).size().reset_index(name="count"))

Total locations: 424
  geocoded=True:        424 (100.0%)
  within_boundary=True: 373 (88.0%)

Breakdown:
   geocoded  within_boundary  count
0      True            False     51
1      True             True    373


## Save Validated Results

In [16]:
# Select columns for output
output_cols = [
    "location_name",
    "geocode_query",
    "subdistrict",
    "district",
    "original",
    "province",
    "constituency_no",
    "Lat",
    "Lng",
    "PlaceId",
    "FormattedAddress",
    "geocoded",
    "within_boundary",
]

output_df = df[output_cols].copy()
output_df.head()

,location_name,geocode_query,subdistrict,district,original,province,constituency_no,Lat,Lng,PlaceId,FormattedAddress,geocoded,within_boundary
0,สำนักงานเขตสัมพันธวงศ์,สำนักงานเขตสัมพันธวงศ์,ตลาดน้อย,สัมพันธวงศ์,บริเวณสำนักงานเขตสัมพันธวงศ์ แขวงตลาดน้อย,กรุงเทพมหานคร,1,13.731548,100.513913,ChIJFS6-G9mY4jART6t02bdSwTc,37 ถนน โยธา แขวงตลาดน้อย เขตสัมพันธวงศ์ กรุงเท...,True,True
1,สำนักงานเขตป้อมปราบศัตรูพ่าย,สำนักงานเขตป้อมปราบศัตรูพ่าย,วัดโสมนัส,None,บริเวณสำนักงานเขตป้อมปราบศัตรูพ่าย แขวงวัดโสมนัส,กรุงเทพมหานคร,1,13.758143,100.513079,ChIJceCxnj-Z4jARzcJ6U54enY0,50 ถ. ศุภมิตร แขวงวัดโสมนัส เขตป้อมปราบศัตรูพ่...,True,True
2,สำนักงานเขตพระนคร,สำนักงานเขตพระนคร,วัดสามพระยา,พระนคร,บริเวณลานจอดรถสำนักงานเขตพระนคร แขวงวัดสามพระยา,กรุงเทพมหานคร,1,13.764888,100.498756,ChIJFZE9h26Z4jARFVWMQbVy6SQ,78 ถ. สามเสน แขวงวัดสามพระยา เขตพระนคร กรุงเทพ...,True,True
3,สมาคมใหหนำแห่งประเทศไทย,บริเวณสโมสรสมาคมใหหนำแห่งประเทศไทย,สี่พระยา,None,บริเวณสโมสรสมาคมใหหนำแห่งประเทศไทย แขวงสี่พระยา,กรุงเทพมหานคร,1,13.726568,100.519681,ChIJaySw6dOY4jARXl9EOXANQMI,324 ซอย สุรวงศ์ แขวงสี่พระยา เขตบางรัก กรุงเทพ...,True,True
4,วชิราวุธวิทยาลัย,วชิราวุธวิทยาลัย,ดุสิต,None,บริเวณภายในวชิราวุธวิทยาลัย แขวงดุสิต,กรุงเทพมหานคร,1,13.776863,100.515750,ChIJd2QIDVeZ4jARpdp_-8_9m7w,197 ถ. ราชวิถี แขวงดุสิต เขตดุสิต กรุงเทพมหานค...,True,True


In [17]:
# Save to parquet
output_path = "../intermediate/early_voting_validated.parquet"
output_df.to_parquet(output_path)
print(f"Saved to {output_path}")

Saved to ../intermediate/early_voting_validated.parquet


## Quick Visualization

In [ ]:
# Create GeoDataFrame for visualization
valid_df = output_df[output_df["Lat"].notna()].copy()
geometry = [Point(lng, lat) for lng, lat in zip(valid_df["Lng"], valid_df["Lat"])]
gdf = gpd.GeoDataFrame(valid_df, geometry=geometry, crs="EPSG:4326")

# Plot
ax = ect.plot(figsize=(12, 12), alpha=0.3, edgecolor="gray")
gdf[gdf["within_boundary"]].plot(
    ax=ax, color="green", markersize=30, label="within_boundary=True"
)
gdf[~gdf["within_boundary"]].plot(
    ax=ax, color="orange", markersize=30, label="within_boundary=False"
)
ax.legend()
ax.set_title("Early Voting Locations - Boundary Validation")